In [4]:
import os
import time
import json
from math import ceil
from pprint import pprint
import requests
import datetime
from dotenv import load_dotenv

load_dotenv()

from vk_photo import VkApiHandler


In [6]:
if __name__ == '__main__':
    vk_token = os.getenv('VK_API_TOKEN')
    vk= VkApiHandler(vk_token, '1', 5)
    # vk = VkApiHandler(vk_token, '1', 5)
    # with open('vk_photo.json', 'w') as outfile:
    #     json.dump(vk.get_photo_info('1'), outfile)
    
    # pprint(vk.get_photo_info())
    # photo_count, photo_items = vk.get_photo_info()
    # print(photo_count)
    # for i in range(photo_count):
    #     url, p_size = vk.find_photo_max_size(photo_items[i]['sizes'])
    #     print(f'{p_size}: {url}')
    # pprint(vk.get_photo_params())
    # jl, ud = vk.get_photo_json()
    # pprint(jl)
    # pprint(ud)
    # with open('vk_photo.json', 'w') as outfile:
    #     json.dump(jl, outfile)
    # with open('vk_photo.json', 'w') as outfile:
    #     json.dump(vk.json, outfile)
        
    # pprint(vk.export_dict)



Получен массив из 5 фотографий. Выполнено 56 %
Получен массив из 9 фотографий. Выполнено 100 %


In [ ]:
class YandexDisk:

    def __init__(self, folder_name, token, num=5):
        self.token = token
        self.file_upload_num = num
        self.url = 'https://cloud-api.yandex.net/v1/disk/resources/upload'
        self.headers = self.get_headers()
        self.folder = self.create_folder(folder_name)

    def get_headers(self):
        return {
            'Content-Type': 'application/json',
            'Authorization': 'OAuth {}'.format(self.token)
        }
    
    def create_folder(self, folder_name):
        """Метод создает папку на Yandex Disk для загрузки фотографий"""
        url = "https://cloud-api.yandex.net/v1/disk/resources"
        params = {'path': folder_name}
        resp = requests.get(url, headers=self.headers, params=params)
        if resp.status_code != 200:
            response = requests.put(url, headers=self.headers, params=params)
            if response.status_code == 201:
                print(f'Папка {folder_name} созджана в корневом каталоге Яндекс диска')
            else:
                print(f'Папке не была создана! Код ошибки: {response.status_code}')
        else:
            print(f'Папка {folder_name} уже существует. Файлы с одинаковыми именами не будут скопированы')
        return folder_name
        



        




    def get_files_list(self):
        files_url = 'https://cloud-api.yandex.net/v1/disk/resources/files'
        headers = self.get_headers()
        response = requests.get(files_url, headers=headers)
        return response.json()

    def _get_upload_link(self, disk_file_path):
        upload_url = "https://cloud-api.yandex.net/v1/disk/resources/upload"
        headers = self.get_headers()
        params = {"path": disk_file_path, "overwrite": "true"}
        response = requests.get(upload_url, headers=headers, params=params)
        return response.json()

    def upload_file_to_disk(self, disk_file_path, filename):
        href = self._get_upload_link(disk_file_path=disk_file_path).get("href", "")
        with open(filename, 'rb') as data:
            response = requests.put(href, data)
            response.raise_for_status()
            if response.status_code == 201:
                print("Success")